In [1]:
#@title Import dependencies. { display-mode: "form" }
# Load ProtT5 in half-precision (more specifically: the encoder-part of ProtT5-XL-U50)
from transformers import T5Tokenizer, T5EncoderModel, BertGenerationEncoder, BertTokenizer
import torch
import re
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("Using device: {}".format(device))

Using device: cuda:0


In [2]:
import torch

if_cuda = torch.cuda.is_available()
print("if_cuda=",if_cuda)
gpu_count = torch.cuda.device_count()
print("gpu_count=",gpu_count)


if_cuda= True
gpu_count= 1


In [3]:
#@title Load encoder-part of ProtT5 in half-precision. { display-mode: "form" }
# Load ProtT5 in half-precision (more specifically: the encoder-part of ProtT5-XL-U50 in half-precision)
transformer_link = "prot_t5_xl_half_uniref50-enc"
print("Loading: {}".format(transformer_link))
model = T5EncoderModel.from_pretrained(transformer_link)
if device==torch.device("cuda"):
  print("Casting model to full precision for running on CPU ...")
  model.to(torch.float32) # only cast to full-precision if no GPU is available
model = model.to(device)
model = model.eval()
tokenizer = T5Tokenizer.from_pretrained(transformer_link, do_lower_case=False, legacy=True )

Loading: prot_t5_xl_half_uniref50-enc


In [59]:
import pandas as pd
sequence_examples = pd.read_csv("dataset/PSI_Biology_solubility_trainset.csv").iloc[:,0].values.tolist()
# sequence_examples = ["PRTEINO", "SEQWENCE"]
# this will replace all rare/ambiguous amino acids by X and introduce white-space between all amino acids
i=0
for seq in sequence_examples:
  # replace all rare/ambiguous amino acids by X and introduce white-space between all amino acids
  print(i)
  i = i+1
  print(seq)
  sequence_examples = [" ".join(list(re.sub(r"[UZOB]", "X", seq))) ]

  # tokenize sequences and pad up to the longest sequence in the batch
  ids = tokenizer(sequence_examples, add_special_tokens=True, padding="longest")

  input_ids = torch.tensor(ids['input_ids']).to(device)
  attention_mask = torch.tensor(ids['attention_mask']).to(device)

  # generate embeddings
  with torch.no_grad():
      embedding_repr = model(input_ids=input_ids, attention_mask=attention_mask)

  # extract residue embeddings for the first ([0,:]) sequence in the batch and remove padded & special tokens ([0,:7])
  emb_0 = embedding_repr.last_hidden_state[0,:len(sequence_examples[0])] # shape (7 x 1024)
  # print(f"Shape of per-residue embedding of first sequences: {emb_0.shape}")
  # same for the second ([1,:]) sequence but taking into account different sequence lengths ([1,:8])
  # if you want to derive a single representation (per-protein embedding) for the whole protein
  emb_0_per_protein = emb_0.mean(dim=0) # shape (1024)
  with open("dataset/PSI_Biology_solubility_trainsetProtBert.csv", 'a') as f:
    # print(emb_0_per_protein)
    a = []
    for j in emb_0_per_protein:
      # print(j)
      j = str(j)
      j = j.replace("tensor(","")
      j = j.replace(", device='cuda:0')","")
      # print(j)
      a.append(float(j))
    f.write(str(a))
    print(len(a))
    f.write("\n")
  # print(f"Shape of per-residue embedding of first sequences: {emb_0_per_protein.shape}")
  # print(emb_0_per_protein)

0
MTYKDGTYSSDGTYTSPNGLETVGVELTLAADKVSAVNITVHPSNPNTKKFQGEFASGIAAQIVGKDIDELNVSKVAGSSLTSGGFNEAVEQIKSQAK
1024
1
MTAMNILVLGSDSRGSSDADVEANTATDQRADTLMLVHVPADRKKTYAISLMRDLWIDIPGVGESKINAALAYGGVPLMVQTVESLLQQRIDHVAMIDFQGFKGLTDALGGVEVDVRIPFAPSSGPKAGQVYDAGKQTLNGDEALAFVRERKAFSDGDYQRVRNQQAYLKAIIGKVIARETLTNPVTISNMVSAVSPFVSVDKDFDAAAIGSLALSMKDVRAQDTVMFTLPTLGTGTSPDGQSIVLADTTAIADVTAALAKDQVDAYITAHALDKGF
1024
2
MKAEGNTAMNILVLGSDSRGSSDADVEANTATDQRADTLMLVHVPADRKKTYAISLMRDLWIDIPGVGESKINAALAYGGVPLMVQTVESLLQQRIDHVAMIDFQGFKGLTDALGGVEVDVRIPFAPSSGPKAGQVYDAGKQTLNGDEALAFVRERKAFSDGDYQRVRNQQAYLKAIIGKVIARETLTNPVTISNMVSAVSPFVSVDKDFDAAAIGSLALSMKDVRAQDTVMFTLPTLGTGTSPDGQSIVLADTTAIADVTAALAKDQVDAYITAHALDKGF
1024
3
MQSFNSGSTKIHNAFPEESTRPQKAEGNTAMNILVLGSDSRGSSDADVEANTATDQRADTLMLVHVPADRKKTYAISLMRDLWIDIPGVGESKINAALAYGGVPLMVQTVESLLQQRIDHVAMIDFQGFKGLTDALGGVEVDVRIPFAPSSGPKAGQVYDAGKQTLNGDEALAFVRERKAFSDGDYQRVRNQQAYLKAIIGKVIARETLTNPVTISNMVSAVSPFVSVDKDFDAAAIGSLALSMKDVRAQDTVMFTLPTLGTGTSPDGQSIVLADTTAIADVTAALAKDQVDAYITAHALDKGF
1024
4
MNAPV

In [23]:
import pandas as pd
sequence_examples = pd.read_csv("dataset/Prics_5foldrest.csv").iloc[:,0].values.tolist()
# sequence_examples = ["PRTEINO", "SEQWENCE"]
# this will replace all rare/ambiguous amino acids by X and introduce white-space between all amino acids
i=0
for seq in sequence_examples:
  # replace all rare/ambiguous amino acids by X and introduce white-space between all amino acids
  print(i)
  i = i+1
  print(seq)
  sequence_examples = [" ".join(list(re.sub(r"[UZOB]", "X", seq))) ]

  # tokenize sequences and pad up to the longest sequence in the batch
  ids = tokenizer(sequence_examples, add_special_tokens=True, padding="longest")

  input_ids = torch.tensor(ids['input_ids']).to(device)
  attention_mask = torch.tensor(ids['attention_mask']).to(device)

  # generate embeddings
  with torch.no_grad():
      embedding_repr = model(input_ids=input_ids, attention_mask=attention_mask)

  # extract residue embeddings for the first ([0,:]) sequence in the batch and remove padded & special tokens ([0,:7])
  emb_0 = embedding_repr.last_hidden_state[0,:len(sequence_examples[0])] # shape (7 x 1024)
  # print(f"Shape of per-residue embedding of first sequences: {emb_0.shape}")
  # same for the second ([1,:]) sequence but taking into account different sequence lengths ([1,:8])
  # if you want to derive a single representation (per-protein embedding) for the whole protein
  emb_0_per_protein = emb_0.mean(dim=0) # shape (1024)
  with open("dataset/Prics_5fold_ProtT5.csv", 'a') as f:
    # print(emb_0_per_protein)
    a = []
    for j in emb_0_per_protein:
      j = str(j)
      j = j.replace("tensor(","")
      j = j.replace(")","")
      # print(j)
      a.append(float(j))
    f.write(str(a))
    f.write("\n")
  # print(f"Shape of per-residue embedding of first sequences: {emb_0_per_protein.shape}")
  # print(emb_0_per_protein)

0
MGKQCFVTGRKASTGNNRSHALNSSKRRWNANLQKVRILVDGKPKKVWVSARALKSGKVTRV


TypeError: tuple indices must be integers or slices, not tuple

In [9]:
import pandas as pd
sequence_examples = pd.read_csv("dataset/Prics_5foldrest.csv").iloc[:,0].values.tolist()
# sequence_examples = ["PRTEINO", "SEQWENCE"]
# this will replace all rare/ambiguous amino acids by X and introduce white-space between all amino acids
i=0
for seq in sequence_examples:
  # replace all rare/ambiguous amino acids by X and introduce white-space between all amino acids
  print(i)
  i = i+1
  print(seq)
  sequence_examples = [" ".join(list(re.sub(r"[UZOB]", "X", seq))) ]

  # tokenize sequences and pad up to the longest sequence in the batch
  ids = tokenizer(sequence_examples, add_special_tokens=True, padding="longest")

  input_ids = torch.tensor(ids['input_ids']).to(device)
  attention_mask = torch.tensor(ids['attention_mask']).to(device)

  # generate embeddings
  with torch.no_grad():
      embedding_repr = model(input_ids=input_ids, attention_mask=attention_mask)

  # extract residue embeddings for the first ([0,:]) sequence in the batch and remove padded & special tokens ([0,:7])
  emb_0 = embedding_repr.last_hidden_state[0,:len(sequence_examples[0])] # shape (7 x 1024)
  # print(f"Shape of per-residue embedding of first sequences: {emb_0.shape}")
  # same for the second ([1,:]) sequence but taking into account different sequence lengths ([1,:8])
  # if you want to derive a single representation (per-protein embedding) for the whole protein
  emb_0_per_protein = emb_0.mean(dim=0) # shape (1024)
  with open("dataset/Prics_5fold_ProtT5.csv", 'a') as f:
    # print(emb_0_per_protein)
    a = []
    for j in emb_0_per_protein:
      j = str(j)
      j = j.replace("tensor(","")
      j = j.replace(")","")
      # print(j)
      a.append(float(j))
    f.write(str(a))
    f.write("\n")
  # print(f"Shape of per-residue embedding of first sequences: {emb_0_per_protein.shape}")
  # print(emb_0_per_protein)

0
MSVHKLTDLRDNSTNWKINVKILSIWNHPPNSHGEITTMILHDDKNNRVDATIPQGNYHNPFCPFLKPGTWIHISDFRVVVPQSRVRYSSFRFHIKFIWETSVHPLPELVKRDFFD
1
ANSIDLIGYVHQPVQFDSTLDGKFWAGTVISHEPSSDSKSESDSSSNFWIPVLFEGDLAHTANSYLKKNDRVHITGQILGDVIQSGANSDQAHVQLFKSFHGSFSHQVMVRD
2
SDSLFLSLAKFDVILSGSTNSNILHDVMGQVIDRSEIQDLNANNKPTKKIDFHLRDQHDTRLACILWGKYAEIVDKACQESTDGIVVCLIRFAKINLYNDTRSVSNFFDVSQVFVDPTLAELDLFKQSIP
3
AAVRKIHVTVKFPSKQFTVEVDRTETVSSLKDKIHIVENTPIKRMQLYYSGIELADDYRNLNEYGITEFSEIVVFLKSINRAKD
4
GDDSVHLHITHANLKSFSADARFSPQMSVEAVKEKLWKKCGTSVNSMALELYDDSGSKVAVLSDDSRPLGFFSPFDGFRLHIIDLDP
5
MAMYIRVKRMKTTYFIQCDPTETVLDVKQKLFALIEQPVNNQRLVLMSTEEVLEDSKSLADQKVENDAVVALTLKKD
6
DPLLVTVITTCGKVFQFHVDRRRNVGYLKKRISKEGKGFPEVDDQEILFKGEKLDDNRIIDGICKDGNSVIHLLVKKS
7
STIKLTVKFGGKSIPLSVSPDCTVKDLKSQLQPITNVLPRGQKLIFKGKVLVETSTLKQSDVGSGAKLMLMASQG
8
MKFLVENLNGSSFELEVDYRDTLLVVKQKIERSQHIPVSKQTLIVDGIVILREDLTVEQCQIVPTSDIQLEVSS
9
MTKQYLTHRCLIAPPEMADDFFANTVIYLARHDDEGAQGLIINRPSGIQVRELLNDLDIEADHVQPHEVLQGGPLRPEAGFVLHTGQPVWHSSIAVGENLCITTSKDILDAIAHNEGVGRYQIALGYASWTKNQLEGE